# Pruebas para crear modulos pipewire

pactl load-module module-null-sink media.class=Audio/Source/Virtual sink_name=my-virtualmic channel_map=front-left,front-right

pactl load-module module-null-sink media.class=Audio/Sink sink_name=mod_out channel_map=stereo

pactl load-module module-null-sink media.class=Audio/Sink sink_name=sine_f_a_p channels=3 channel_map=aux0,aux1,aux2


In [8]:
import sounddevice as sd
import pulsectl as pactl
from pysonance.const import *
 
def callback(indata, frames, time, status):
    if status:
        print(status)
    # Procesar los datos de audio aquí
    print(indata)  # Por ejemplo, imprimir los datos de audio entrantes

In [10]:
for i, dev in enumerate(sd.query_devices()):
    if dev['max_input_channels'] > 0:
        print(i, dev['name'])

5 HD-Audio Generic: ALC887-VD Analog (hw:1,0)
7 HD-Audio Generic: ALC887-VD Alt Analog (hw:1,2)
9 jack
10 pipewire
11 pulse
12 default
13 Starship/Matisse HD Audio Controller Analog Stereo
14 3-Band Splitter
15 spotify
16 speech-dispatcher-dummy
17 Navi 10 HDMI Audio Digital Stereo (HDMI 5)
18 foo Audio/Sink sink
19 foo Audio/Sink sink-102
20 prueba_1
21 prueba_1 Audio/Source/Virtual sink
22 prueba_1-117
23 prueba_2
24 prueba_3


In [11]:
cli = pactl.Pulse('pysonance')

In [12]:
name = 'prueba_4'
_args=[
    'media.class=Audio/Source/Virtual',
    f'sink_name={name}',
    # 'channel_map',
    # f'client_name={self.cli_name}',
    f'rate={SRATE}',
    'channels=1',
    'channel_map=mono',
    f'sink_properties=device.description={name}',
]
cli.module_load('module-null-sink', args=_args)

536870923

In [14]:
sd._terminate()
sd._initialize()
in_stream = sd.InputStream(samplerate=SRATE, channels=1, callback=callback,blocksize=CHUNK, device=f'{name}')


In [15]:
for i, dev in enumerate(sd.query_devices()):
    if dev['max_input_channels'] > 0:
        print(i, dev['name'])

5 HD-Audio Generic: ALC887-VD Analog (hw:1,0)
7 HD-Audio Generic: ALC887-VD Alt Analog (hw:1,2)
9 jack
10 pipewire
11 pulse
12 default
13 Starship/Matisse HD Audio Controller Analog Stereo
14 3-Band Splitter
15 spotify
16 speech-dispatcher-dummy
17 Navi 10 HDMI Audio Digital Stereo (HDMI 5)
18 foo Audio/Sink sink
19 foo Audio/Sink sink-102
20 prueba_1
21 prueba_1 Audio/Source/Virtual sink
22 prueba_1-117
23 prueba_2
24 prueba_3
25 prueba_4


Es necesario finalizar y reiniciar sounddevice para que reconozca los nuevos dispositivos creados con pipewire

In [19]:
cli.module_unload(536870923)

In [3]:
in_stream = sd.InputStream(samplerate=SRATE, channels=1, device='prueba_2',blocksize=CHUNK, callback=callback)


In [ ]:
import subprocess
_id = in_stream._device
print(_id)

AttributeError: 'InputStream' object has no attribute '_stream'

In [4]:
print(sd.query_devices())

   0 HDA ATI HDMI: 0 (hw:0,3), ALSA (0 in, 8 out)
   1 HDA ATI HDMI: 1 (hw:0,7), ALSA (0 in, 8 out)
   2 HDA ATI HDMI: LG FULL HD (hw:0,8), ALSA (0 in, 2 out)
   3 HDA ATI HDMI: 3 (hw:0,9), ALSA (0 in, 8 out)
   4 HDA ATI HDMI: 5 (hw:0,11), ALSA (0 in, 8 out)
   5 HD-Audio Generic: ALC887-VD Digital (hw:1,1), ALSA (0 in, 2 out)
   6 HD-Audio Generic: ALC887-VD Alt Analog (hw:1,2), ALSA (2 in, 0 out)
   7 hdmi, ALSA (0 in, 8 out)
   8 jack, ALSA (2 in, 2 out)
   9 pipewire, ALSA (64 in, 64 out)
  10 pulse, ALSA (32 in, 32 out)
* 11 default, ALSA (32 in, 32 out)
  12 Starship/Matisse HD Audio Controller Analog Stereo, JACK Audio Connection Kit (4 in, 2 out)
  13 3-Band Splitter, JACK Audio Connection Kit (6 in, 2 out)
  14 spotify, JACK Audio Connection Kit (2 in, 0 out)
  15 speech-dispatcher-dummy, JACK Audio Connection Kit (2 in, 0 out)
  16 PortAudio, JACK Audio Connection Kit (1 in, 0 out)
  17 loopback-2249-13, JACK Audio Connection Kit (3 in, 2 out)
  18 loopback-2249-14, JACK Aud

In [8]:
subprocess.run([
    "pactl", "set-sink-property", "11",
    "device.description=NOMBRE NUEVO"
])

No valid command specified.


CompletedProcess(args=['pactl', 'set-sink-property', '11', 'device.description=NOMBRE NUEVO'], returncode=1)